In [2]:
import torch
import numpy as np
import math
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torchvision.transforms as transforms
import torchvision.datasets.voc as VOC


#image transforms
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

#standard libraries
from engine import train_one_epoch, evaluate
import utils
import transforms as T


#our dataset file
from dataset import FruitDataset

let's see if I have cuda on my desktop:

In [4]:
torch.cuda.is_available()

True

graphics card specifications

In [5]:
!nvidia-smi

Mon Apr  5 17:35:45 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 106...  Off  | 00000000:01:00.0  On |                  N/A |
|  0%   43C    P8     6W / 120W |    544MiB /  6075MiB |      6%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [10]:
train_dir = 'data/train_zip/train/'
test_dir = 'data/test_zip/test/'


# Viewing the VOC dataset



Let's create our train dataset and view some files in it:

In [11]:
train_data = FruitDataset(train_dir, 224, 224)
print("here's dataset length: ", len(train_data))

NameError: name '_' is not defined